In [38]:
import pandas as pd
import os

In [40]:
data = pd.read_stata(os.getcwd() + '/data/Info_by_metrics.dta')
data1 = pd.read_stata(os.getcwd() + '/data/analysis.dta')
data2 = pd.read_stata(os.getcwd() + '/data/Rug_Surveys.dta')


/var/folders/zn/gw_srj0x2d9c3gpr6_vc6vvr0000gn/T/ipykernel_55710/1356626342.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  data1 = pd.read_stata(os.getcwd() + '/data/analysis.dta')


In [41]:
data.to_csv(os.getcwd() + '/data/analysis.csv', index=False)
data1.to_csv(os.getcwd() + '/data/Info_by_metrics.csv', index=False)
data2.to_csv(os.getcwd() + '/data/Rug_Surveys.csv', index=False)

In [42]:
data = pd.read_csv(os.getcwd() + '/data/analysis.csv')
data

,id,info_metric,info_get_better,info_technique,info_was_taught,strata,treatment
0,336,Packedness,1,1.0,1.0,2.0,1.0
1,336,Corners,2,NaN,0.0,2.0,1.0
2,336,Waviness,1,1.0,1.0,2.0,1.0
3,336,Weight,1,1.0,1.0,2.0,1.0
4,336,Touch,2,NaN,0.0,2.0,1.0
...,...,...,...,...,...,...,...
1477,343,Loom Tightness,2,NaN,0.0,4.0,1.0
1478,343,Design Accuracy,2,NaN,0.0,4.0,1.0
1479,343,Sidda Packedness,2,NaN,0.0,4.0,1.0
1480,343,Design Ability,2,NaN,0.0,4.0,1.0


In [36]:
data1 = pd.read_csv(os.getcwd() + '/data/Info_by_metrics.csv')
data1

/var/folders/zn/gw_srj0x2d9c3gpr6_vc6vvr0000gn/T/ipykernel_55710/1092000174.py:1: DtypeWarning: Columns (20,67) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv(os.getcwd() + '/data/Info_by_metrics.csv')


,round,id,strata,treatment,age_resp,digit_span_recall1,typical_earnings_member1,typical_earnings_member2,typical_earnings_member3,typical_earnings_member4,...,qual_design_accuracy_adj3_b,qual_design_accuracy_adj4_b,qual_sidda_packed_adj1_b,qual_sidda_packed_adj2_b,qual_sidda_packed_adj3_b,qual_sidda_packed_adj4_b,m2_per_hour_direct_adj2_b,m2_per_all_input_adj2_b,hhsize,control_mean
0,100.0,1.0,7.0,1.0,55.0,4.0,1000.0,NaN,NaN,NaN,...,0.367984,0.033559,0.0,0.0,0.0,0.0,0.0,11.018080,6.0,1.0
1,102.0,1.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.367984,0.033559,0.0,0.0,0.0,0.0,0.0,11.018080,NaN,1.0
2,103.0,1.0,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.367984,0.033559,0.0,0.0,0.0,0.0,0.0,11.018080,NaN,1.0
3,110.0,1.0,7.0,1.0,61.0,NaN,850.0,NaN,NaN,NaN,...,0.367984,0.033559,0.0,0.0,0.0,0.0,0.0,11.018080,3.0,1.0
4,100.0,2.0,6.0,1.0,41.0,4.0,600.0,NaN,NaN,NaN,...,-1.887538,-1.080055,0.0,0.0,0.0,0.0,0.0,0.100659,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,201.0,1585.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2347,202.0,1591.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2348,210.0,1595.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0
2349,210.0,1596.0,NaN,NaN,40.0,NaN,1680.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0


In [43]:
data2 = pd.read_csv(os.getcwd() + '/data/Rug_Surveys.csv')
data2

,id,qual_sidda,qual_inputs,qual_packedness,qual_loom_tight,qual_size_accuracy,qual_design_accuracy,qual_finishing_time,qual_overall,date,...,produced104,produced105,produced110,produced201,produced202,produced203,weight1,weight2,weight_error,abs_weight_error
0,22.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,2014-01-10,...,424.9021,838.40210,532.9021,502.9021,718.4021,934.4021,2.5,2.4,-0.040000,0.040000
1,22.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,2013-11-15,...,424.9021,838.40210,532.9021,502.9021,718.4021,934.4021,2.5,2.3,-0.080000,0.080000
2,22.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,NaN,2012-10-19,...,424.9021,838.40210,532.9021,502.9021,718.4021,934.4021,2.4,2.5,0.041667,0.041667
3,22.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,2012-12-21,...,424.9021,838.40210,532.9021,502.9021,718.4021,934.4021,2.5,2.4,-0.040000,0.040000
4,22.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,2013-12-13,...,424.9021,838.40210,532.9021,502.9021,718.4021,934.4021,2.5,2.3,-0.080000,0.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,684.0,5.0,5.0,4.0,4.0,3.0,3.0,3.0,NaN,2013-10-04,...,NaN,439.19998,79.2000,NaN,252.0000,639.3600,2.5,2.1,-0.160000,0.160000
3825,684.0,5.0,5.0,4.0,4.0,3.0,4.0,3.0,NaN,2013-07-05,...,NaN,439.19998,79.2000,NaN,252.0000,639.3600,2.5,2.1,-0.160000,0.160000
3826,684.0,5.0,5.0,4.0,4.0,3.0,3.0,3.0,NaN,2014-01-10,...,NaN,439.19998,79.2000,NaN,252.0000,639.3600,2.5,2.1,-0.160000,0.160000
3827,684.0,5.0,5.0,4.0,4.0,3.0,3.0,3.0,NaN,2013-12-06,...,NaN,439.19998,79.2000,NaN,252.0000,639.3600,2.5,2.1,-0.160000,0.160000
